In [32]:
import argparse
import os
import sys
import tabulate
import time
import torch
import torch.nn.functional as F

import curves
import data
import models
import utils
import pickle

import copy

In [33]:
architecture = getattr(models, "VGG16")

In [34]:
architecture

models.vgg.VGG16

In [35]:
model = architecture.base(num_classes=10, **architecture.kwargs)

In [36]:
# list(model.modules())

In [37]:
# model.load_state_dict(torch.load("curve/checkpoint-50.pt")['model_state'])

In [38]:
# torch.load("curve/checkpoint-50.pt")['model_state']

In [39]:
# list(model.modules())

In [40]:
# python train.py --dir=curve --model=VGG16 --data_path=data --epochs=100

In [41]:
# args = parser.parse_args(['--dir=curve'])

In [42]:
# loaders, num_classes = data.loaders(
#     args.dataset,
#     args.data_path,
#     args.batch_size,
#     args.num_workers,
#     args.transform,
#     args.use_test
# )

In [43]:
loaders, num_classes = data.loaders(
    "CIFAR10",
    "data",
    128,
    1,
    "VGG",
    False
)

Files already downloaded and verified
Using train (45000) + validation (5000)
Files already downloaded and verified


In [44]:
for X, y in loaders['test']:
    break

In [45]:
X.shape

torch.Size([128, 3, 32, 32])

In [46]:
y_pred = torch.argmax(model(X), dim=-1)

In [47]:
y

tensor([ 7,  1,  4,  1,  0,  2,  2,  5,  9,  6,  0,  1,  3,  1,
         1,  2,  1,  8,  5,  6,  9,  2,  4,  2,  8,  8,  3,  7,
         1,  7,  1,  7,  5,  9,  7,  0,  7,  8,  3,  6,  1,  2,
         2,  5,  9,  8,  4,  8,  9,  4,  2,  5,  9,  3,  7,  9,
         9,  3,  7,  9,  2,  7,  8,  9,  0,  7,  3,  0,  1,  7,
         1,  4,  6,  8,  2,  3,  4,  1,  6,  2,  6,  0,  3,  5,
         1,  0,  9,  7,  0,  0,  9,  3,  4,  9,  8,  8,  0,  1,
         6,  1,  4,  9,  0,  7,  8,  1,  0,  9,  5,  2,  4,  4,
         7,  0,  4,  8,  9,  3,  1,  2,  2,  4,  4,  0,  4,  5,
         1,  7])

In [48]:
y_pred

tensor([ 8,  8,  3,  3,  8,  3,  3,  8,  3,  8,  3,  4,  3,  3,
         3,  3,  8,  3,  8,  3,  3,  8,  3,  8,  8,  8,  3,  3,
         8,  3,  8,  3,  3,  3,  3,  3,  3,  3,  3,  8,  3,  3,
         3,  3,  3,  3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         8,  3,  3,  3,  3,  3,  8,  3,  3,  3,  3,  8,  3,  3,
         4,  3,  3,  8,  3,  3,  3,  3,  8,  3,  3,  3,  3,  3,
         3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  8,  3,  8,  3,  3,  3,  3,  3,  3,  3,  3,  3,  8,
         3,  4,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,
         3,  3])

In [49]:
model

VGGBase(
  (layer_blocks): ModuleList(
    (0): ModuleList(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (1): ModuleList(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (2): ModuleList(
      (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (3): ModuleList(
      (0): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    )
    (4): ModuleList(
      (0): Conv2d(512, 512, kernel_size=(3, 3), 

## Rescale

In [50]:
def rescale(l, scale):
    list(model.modules())[l].weight=torch.nn.Parameter(list(model.modules())[l].weight*scale)
    list(model.modules())[l].bias=torch.nn.Parameter(list(model.modules())[l].bias*scale)

In [58]:
list(model.modules())[18]

Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [52]:
rescale(-3, 10)

In [53]:
rescale(-6, 0.1)

In [54]:
y_pred_r = torch.argmax(model(X), dim=-1)

In [55]:
y_pred_r

tensor([ 8,  8,  8,  8,  8,  3,  8,  3,  3,  8,  8,  3,  8,  8,
         3,  3,  8,  3,  8,  8,  3,  8,  8,  8,  8,  8,  8,  3,
         8,  8,  8,  3,  3,  8,  8,  8,  8,  3,  8,  8,  8,  8,
         8,  3,  3,  8,  8,  3,  8,  8,  8,  3,  8,  8,  8,  8,
         8,  8,  8,  3,  8,  8,  8,  8,  8,  8,  8,  3,  8,  8,
         8,  8,  3,  3,  8,  8,  8,  8,  8,  3,  3,  8,  8,  8,
         3,  3,  8,  8,  8,  8,  3,  8,  8,  8,  8,  3,  3,  8,
         8,  3,  3,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  3,  3,  8,  8,
         3,  8])

In [56]:
sum(~(y_pred==y_pred_r))

tensor(83, dtype=torch.uint8)

## Node

In [53]:
def change_node(l1, l2, i, j):
    
    c = copy.deepcopy(torch.nn.Parameter(list(model.modules())[l1].weight[j]))
    list(model.modules())[l1].weight[j]  = list(model.modules())[l1].weight[i] 
    list(model.modules())[l1].weight[i] = c
    
    c = copy.deepcopy(torch.nn.Parameter(list(model.modules())[l2].weight.transpose(0,1)[j]))
    list(model.modules())[l2].weight.transpose(0,1)[j]  = list(model.modules())[l2].weight.transpose(0,1)[i]
    list(model.modules())[l2].weight.transpose(0,1)[i] = c
    
 

In [54]:
change_node(-3, -1, 0, 1)

In [55]:
y_pred_n = torch.argmax(model(X), dim=-1)

In [56]:
y_pred_n

tensor([ 8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8])

In [62]:
print("eq ", ~(y_pred_n==y_pred_r))

eq  tensor([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0], dtype=torch.uint8)


## Saving

In [71]:
#  torch.load("curve/checkpoint-50.pt")

In [74]:
for path, k in [("Aaa", 0), ("Bbb", 4 - 1)]:
    print('p ', path)
    print('k', k)
    

p  Aaa
k 0
p  Bbb
k 3


In [77]:
a = list(range(10))
a[::3]

[0, 3, 6, 9]

In [89]:
architecture = getattr(models, "VGG16")
curve = getattr(curves, 'PolyChain')

In [90]:
curve

curves.PolyChain

In [86]:
architecture.curve

models.vgg.VGGCurve

In [87]:
architecture.kwargs

{'batch_norm': False, 'depth': 16}

In [113]:
model = curves.CurveNet(
        10,
        curve,
        architecture.curve,
        3,
        True,
        True,
        architecture_kwargs=architecture.kwargs,
    )

In [114]:
model

CurveNet(
  (coeff_layer): PolyChain()
  (net): VGGCurve(
    (layer_blocks): ModuleList(
      (0): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
      )
      (1): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
      )
      (2): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
        (2): Conv2d()
      )
      (3): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
        (2): Conv2d()
      )
      (4): ModuleList(
        (0): Conv2d()
        (1): Conv2d()
        (2): Conv2d()
      )
    )
    (activation_blocks): ModuleList(
      (0): ModuleList(
        (0): ReLU(inplace)
        (1): ReLU(inplace)
      )
      (1): ModuleList(
        (0): ReLU(inplace)
        (1): ReLU(inplace)
      )
      (2): ModuleList(
        (0): ReLU(inplace)
        (1): ReLU(inplace)
        (2): ReLU(inplace)
      )
      (3): ModuleList(
        (0): ReLU(inplace)
        (1): ReLU(inplace)
        (2): ReLU(inplace)
      )
      (4): ModuleList(
        (

In [94]:
pred = model(X)

In [95]:
pred.shape

torch.Size([128, 10])

In [96]:
from torch.nn import Module

In [98]:
m = Module()

In [105]:
m.register_buffer('range', torch.arange(0, float(3)))

In [106]:
m.range

tensor([ 0.,  1.,  2.])

In [122]:
t_n = 2/3 * (3 - 1)
torch.max(m.range.new([0.0]), 1.0 - torch.abs(t_n - m.range))

tensor([ 0.0000,  0.6667,  0.3333])